# Testing OGC CSW search against the CVL catalogue

**Warning: still working on the end point setup and content**

The purpose of this Notebook is to show how the machine end point for the CVL catalogue can be queried for relevant datasets. This setup can be integrated into a more comprehensive Notebook or Python programme.

Some relevant information on how to utilise OGC CSW is provided at e.g. https://gist.github.com/kalxas/6ecb06d61cdd487dc7f9 or https://ioos.github.io/ioos_code_lab/content/code_gallery/data_access_notebooks/2016-12-19-exploring_csw.html. Details on the features of the software used for the endpoint (pyCSW) is provided at https://pycsw.org/ with a FAQ at https://pycsw.org/faq/. Some simple examples of how to use OWSLib for this purpose is provided in https://geopython.github.io/OWSLib/usage.html#csw.

First we have to import OWSLib as we use this to set up the query.

In [1]:
from owslib.csw import CatalogueServiceWeb

Set up connection to the end point for CVL services.

In [66]:
csw = CatalogueServiceWeb('https://cvl.csw.met.no/csw')

Import further elements from OWSLib that are required below.

In [79]:
from owslib.fes import PropertyIsLike, PropertyIsLessThanOrEqualTo, PropertyIsGreaterThanOrEqualTo, BBox, And

First a general query for records containing the Sentinel-2 is done. We search for data north of 50N, but south of 90N and east of 0E, but west of 25E and only data for the period 2020-02-01 - 2020-06-30. These filters are combined using the operator "And" to ensure that all filters are applied when searching.

Concerning the Bounding Box, the specification of this is a bit counter intuitive (at least for many) in the sense that it follows the approach of lower left and upper right corners specified using latitude, longitude order (i.e. Y then X).

If results are required in the response, add resulttype=results to the request.

The default output from OWSLib is Dublin Core, to make sure a proper document is returned, add outputschema as indicated below. then you will get a full ISO19115 record.

In [80]:
# BBox lat1, lon1, lat2, lon2
mybboxq = BBox([50,0,90,25],crs="urn:ogc:def:crs:EPSG:4326")
mytitleq = PropertyIsLike('apiso:Abstract','%SENTINEL-2%')
mystart = PropertyIsGreaterThanOrEqualTo('apiso:TempExtent_begin','2020-02-01')
myend = PropertyIsLessThanOrEqualTo('apiso:TempExtent_end','2020-06-30')
myfilters = [And([mytitleq, mybboxq, mystart, myend])]

csw.getrecords2(constraints=myfilters, maxrecords=20, outputschema="http://www.isotc211.org/2005/gmd")

The results are shown below. It is evident that 510 records (datasets) contain this word in the title.

In [77]:
csw.results

{'matches': 0, 'returned': 0, 'nextrecord': 0}

As only the list is returned, we need to loop through this and check the contents, e.g. by listing identifiers and title.

In [81]:
for rec in csw.records:   
    print(rec)
    print(csw.record[rec].xml)

In order to check a specific record, retrieve its content using DescribeRecord.